In [40]:
import pandas as pd
import numpy as np
import re
import string
import unicodedata

In [41]:
df  = pd.read_csv('comentarios_completo.csv')

Filtrar calificaciones dobles

In [42]:
display(df['calificacion'].unique())
calif_dobles = df['calificacion'].astype(str).str.contains(' ')
df.loc[calif_dobles, 'calificacion'] = df.loc[calif_dobles, 'calificacion'].astype(str).str.split().str[0]
df['calificacion'].unique()

array(['6', nan, '10', '9', '8', '8 10', '6 10 ', '8 y 10', '8 y 10 ',
       ' ', '4', '0', '7', '1', '5', '6.0', '10.0', '8.0', '2', '10 y 6 ',
       '0.0', '9.0', '3', '7.0'], dtype=object)

array(['6', nan, '10', '9', '8', '4', '0', '7', '1', '5', '6.0', '10.0',
       '8.0', '2', '0.0', '9.0', '3', '7.0'], dtype=object)

In [43]:
df['calificacion'] = pd.to_numeric(df['calificacion'], errors='coerce').astype('Int8') 
df = df.dropna(subset=['calificacion'])
df.iloc[50:56]

,calificacion,comentarios
128,8,"Solo califica, sin comentarios"
130,6,"Solo califica, sin comentarios"
131,10,"Solo califica, sin comentarios"
133,10,"Solo califica, sin comentarios"
136,10,"Solo califica, sin comentarios"
140,6,Se tardan mucho para recibir y entregar a pesa...


Limpiar texto

In [ ]:
def quitar_acentos(texto: str) -> str:
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join([c for c in texto if unicodedata.category(c) != 'Mn'])
    return texto

def limpiar(texto: str):
    if pd.isna(texto) or texto is None:
        return np.nan

    if not isinstance(texto, str) or texto.strip() == '' or re.sub(f"[{re.escape(string.punctuation)}]", '', texto) == '':
        return np.nan

    if('*' in texto or '#' in texto):
        return '*'
    
    # Cambiar todo a minusculas
    texto = texto.lower()
    # Eliminar acentos
    texto = quitar_acentos(texto)
    # Eliminar signos de puntuacion
    #texto = re.sub('[%s]' % re.escape(string.punctuation), ' ', texto)
    # Eliminar saltos de linea
    texto = re.sub(r'\n', ' ', texto)
    # Eliminar espacios seguidos
    texto = re.sub(r'\s+', ' ', texto)
    # Eliminar espacios al inicio y al final
    texto = texto.strip()

    return texto

df['comentarios'] = df['comentarios'].astype('string')
df['comentarios'] = df['comentarios'].apply(limpiar)
df = df[(df['comentarios'].notna()) & (df['comentarios'].str.strip() != '')]

In [45]:
df.head(20)

,calificacion,comentarios
0,6,pesimo servicio en la agencia
12,10,"solo califica, sin comentarios"
14,9,nos tardaron mas de 1 hora en recibir y no nos...
16,10,"solo califica, sin comentarios"
17,10,"solo califica, sin comentarios"
19,10,"solo califica, sin comentarios"
22,10,"solo califica, sin comentarios"
23,10,"solo califica, sin comentarios"
25,6,"solo califica, sin comentarios"
26,10,"solo califica, sin comentarios"


Buscar comentarios irrelevantes

In [46]:
comentarios_irrelevantes = df[df['comentarios'].str.len() < 40]
comentarios_irrelevantes.to_csv('comentarios_irrelevantes.csv', index=False, encoding='utf-8')
comentarios_irrelevantes.head(n=20)

,calificacion,comentarios
0,6,pesimo servicio en la agencia
12,10,"solo califica, sin comentarios"
16,10,"solo califica, sin comentarios"
17,10,"solo califica, sin comentarios"
19,10,"solo califica, sin comentarios"
22,10,"solo califica, sin comentarios"
23,10,"solo califica, sin comentarios"
25,6,"solo califica, sin comentarios"
26,10,"solo califica, sin comentarios"
27,10,"solo califica, sin comentarios"


Eliminar comentarios irrelevantes

In [47]:
filtro_comentarios = ['*', 'solo califica sin comentarios 1cm', 'no brinda mas comentarios', 'perfecto', 'ninguno gracias', 'en', 'todo muy bien / hyp', 'solo califica sin comentarios/ 1cm', '5cm/ solo califica sin comentarios', 'ninguna.', 'puntualidad', 'solo califica sin comentarios 3cm', 'solo califica sin comentaros',  'solo califica sin comentarios 4cm', 'no proporciona datos ni quiso contestar encuesta', 'ninguna', 'solo califica sin comentarios/ 4cm', 'la rapidez', 'mantener el buen servicio', 'todo ok', 'buen servicio', 'todo muy bien,me trataron bien', 'solo valifica sin comentarios', 'sin comentarios .', '5cm', 'ninguna no quiso proporcionar num celular', 'refacciones', 'no, no hubo comentarios', 'no tiene comentarios', 'ninguno en esta visita', 'todo bien/ correo incorrecto', 'solo califica, sin comentarios', 'todo bien / 5cm', 'sin comentarios', 'atencion', 'solo caifica sin comentarios/ 5cm', 'solo califica sin comentarios 5cm', 'todo bien en todos los aspectos', 'se envía whatsapp', 'no, ninguna', 'no le contestaron el teléfono',  'todo excelente/ 5cm', 'ningún comentario', 'solo califica sin comentarios/ 3cm', 'solo califica sin comentarios/ 5cm', 'solo califica sin comentarios', 'ninguno, gracias', 'no quiso proporcionar num celular', 'no, si lo recomendaría mucho', 'asi de rapidos', 'todo bien, gracias', 'se envia whatsapp', 'no quiso responder encuesta', 'todo bien/ 5cm', 'no brinda conentario', '5cm solo califica sin comentarios', 'todo está muy bien', 'no ninguna', 'nada', 'solo califica sin comentarios / 5cm', 'sin comenatarios', '4cm solo califica sin comentarios', 'no quiso contestar encuesta', 'no', 'no contesta encuesta', 'bdc', 'ninguno']

df = df[~df['comentarios'].isin(filtro_comentarios)]

Guardar

In [48]:
df['calificacion'] = pd.to_numeric(df['calificacion'], errors='coerce').astype('int8')
df['comentarios'] = df['comentarios'].astype('string')
df.to_csv('comentarios_limpios.csv', index=False, encoding='utf-8')